In [61]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix,accuracy_score
#import lightgbm as lgbm
import numpy as np 

In [62]:
train=pd.read_csv('C:/Users/82107/Desktop/open data/병원/train.csv', index_col=0)
test=pd.read_csv('C:/Users/82107/Desktop/open data/병원/test.csv', index_col=0)
submission=pd.read_csv('C:/Users/82107/Desktop/open data/병원/submission_sample.csv', index_col=0)  
print(train.shape)
print(test.shape)
print(submission.shape)

(301, 57)
(127, 57)
(127, 1)


In [63]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301 entries, 1 to 428
Data columns (total 57 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   OC                  301 non-null    object 
 1   sido                301 non-null    object 
 2   sgg                 301 non-null    int64  
 3   openDate            301 non-null    int64  
 4   bedCount            296 non-null    float64
 5   instkind            300 non-null    object 
 6   revenue1            293 non-null    float64
 7   salescost1          293 non-null    float64
 8   sga1                293 non-null    float64
 9   salary1             293 non-null    float64
 10  noi1                293 non-null    float64
 11  noe1                293 non-null    float64
 12  interest1           293 non-null    float64
 13  ctax1               293 non-null    float64
 14  profit1             293 non-null    float64
 15  liquidAsset1        293 non-null    float64
 16  quickAss

In [64]:
train['instkind'].unique()

array(['nursing_hospital', 'general_hospital', 'hospital',
       'traditional_clinic', 'clinic', 'traditional_hospital',
       'dental_clinic', nan], dtype=object)

In [65]:
train["instkind"].value_counts()

nursing_hospital        144
hospital                 88
general_hospital         37
clinic                   18
traditional_hospital      8
traditional_clinic        4
dental_clinic             1
Name: instkind, dtype: int64

In [66]:
print(train.isnull().sum())
#결측치

OC                     0
sido                   0
sgg                    0
openDate               0
bedCount               5
instkind               1
revenue1               8
salescost1             8
sga1                   8
salary1                8
noi1                   8
noe1                   8
interest1              8
ctax1                  8
profit1                8
liquidAsset1           8
quickAsset1            8
receivableS1           8
inventoryAsset1        8
nonCAsset1             8
tanAsset1              8
OnonCAsset1            8
receivableL1           8
debt1                  8
liquidLiabilities1     8
shortLoan1             8
NCLiabilities1         8
longLoan1              8
netAsset1              8
surplus1               8
revenue2               8
salescost2             8
sga2                   8
salary2                8
noi2                   8
noe2                   8
interest2              8
ctax2                  8
profit2                8
liquidAsset2           8


### 내가 해야될거는 회계자료 데이터 전처리
### 따라서 회계자료 결측값은 병원종류(instkind)마다 평균값을 넣어주기로 하자

* 이익잉여금 : 당기순이익은 이익잉여금을 증가시키고 당기순손실은 이익잉여금을 감소킨다
* 매출액
* 매출원가 : 매출액 - 매출원가 = 영업이익
* 판매비, 관리비 : 예로는 급여, 복리후생비, 광고비, 접대비 등
* 급여
* 영업외수익 : 기업의 주된 영업활동이 아닌 활동으로부터 발생한 수익
* 영업외비용 : 기업의 주된 영업활동이 아닌 활동으로부터 발생한 비용(이자비용 포함)
* 이자비용
* 법인세비용 : 엽업이익 - 법인세비용 = 순이익
* 당기순이익 : 당기순이익 = 영업이익 + 영업외수익 - 영업외비용 - 법인세
* 유동자산 : 당좌, 재고자산 포함 => 다른 재화(財貨)와 쉽게 바꿀 수 있을 뿐 아니라 다른 재산에 비하면 그 가치가 안정
* 당좌자산
* 미수금(단기) : ?
* 재고자산

### 그리고 회계자료에서 분석해야될 것
1. 16-17 매출 변화량 => revenue1 / revenue2
2. 유동자산 변화량(유동자산이 크면 안정적인 회사) => liquidAsset1 – liquidAsset2
3. 16년 총이익 => revenue2 - salescost2
4. 16년 영업 이익 => (revenue2 - salescost2) - sga2
5. 16년 영업 이익률(8%~22% 사이면 좋은 회사) => (revenue2 - salescost2)/revenue2
6. 16년 당기순이익 => ((revenue2 - salescost2) - sga2) + noi2 - noe2 - ctax2
7. 16년 당기순이익/영업이익 => Profit2/(revenue2 - salescost2)
* 결국 변화량이 중요한거니까 17년도꺼도 해야됨

In [67]:
train

,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
inst_id,,,,,,,,,,,,,,,,,,,,,
1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,0.000000e+00,3.961135e+09,2.033835e+09,...,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,same
3,open,gyeongnam,32,19970401,410.0,general_hospital,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,801.0,813.0,same
4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,same
7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,0.000000e+00,7.067740e+10,3.178605e+10,...,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,same
9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,0.000000e+00,4.765605e+10,2.446078e+10,...,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,same
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,open,jeonnam,178,20050211,214.0,nursing_hospital,7.614697e+09,4.153475e+08,5.903119e+09,3.467861e+09,...,9.423771e+09,2.618201e+09,0.000000e+00,6.805570e+09,5.930000e+09,5.379502e+09,8.116416e+08,193.0,141.0,same
423,close,choongnam,159,20140618,NaN,hospital,6.717144e+09,8.200000e+09,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,79.0,79.0,same
426,close,gyeongbuk,107,19830705,NaN,hospital,0.000000e+00,0.000000e+00,5.479428e+07,7.500000e+06,...,6.370097e+09,2.170973e+09,0.000000e+00,4.199124e+09,2.498749e+09,3.259161e+09,0.000000e+00,NaN,NaN,NaN


In [68]:
#결측치 채우기(각 열의 평균값)
#위에서 확인했듯 회계자료 모든 열이 같은 행에서 결측치를 가짐
#생각해보니 회계 자료 말고도 결측값이 존재, 코드 합칠때 다른 열부터 결측값 처리하고 평균값으로 바꾸자

#columns = train.columns[6:55].tolist()

#for i in range(len(columns)):
#    train[columns[i]].fillna(train.groupby('instkind')[columns[i]].transform('mean'), inplace=True)
    
#for i in range(len(columns)):
#    if train[columns[i]]==0:
#        train[columns[i]].fillna(train.groupby('instkind')[columns[i]].transform('mean'), inplace=True)

#0값이 있으니까 뒤에 새로 만든 열을 생성할때 nan값이 생성, 해결하려 했으나 포기,,,

In [69]:
#인터넷에서 퍼온 코드
def fill_the_missing(df):
    columns = df.columns[6:55].tolist()
    
    for i in range(len(columns)):
        df[columns[i]] = df[columns[i]].fillna(df.groupby('instkind')[columns[i]].transform('mean'))

    for i in range(len(columns)):
        df[columns[i]] = df[columns[i]].apply(lambda x: x+1)

In [70]:
fill_the_missing(train)

In [75]:
print(train.isnull().sum())
#결측치 없어진거 확인

OC                    0
sido                  0
sgg                   0
openDate              0
bedCount              5
                     ..
17_n                  0
OPM_change            0
16_liquid             0
17_liquid             0
liquidAsset_change    0
Length: 64, dtype: int64


In [76]:
def new_column(data):
    #매출액 변화량
    data['revenue_change'] = data['revenue1'] - data['revenue2']
    
    #영업 이익률
    data['16_n'] = data['revenue2'] - data['salescost2']
    data['17_n'] = data['revenue1'] - data['salescost1']
    #영업 이익률 변화
    data['OPM_change'] = data['17_n'] - data['16_n']
    
    # 유동자산 비율
    data["16_liquid"] = data["liquidAsset2"] / (data["liquidAsset2"] + data["nonCAsset2"])
    data["17_liquid"] = data["liquidAsset1"] / (data["liquidAsset1"] + data["nonCAsset1"])
    # 유동자산비율 변화
    data["liquidAsset_change"] = data["17_liquid"] - data["16_liquid"]
    
    return data

In [77]:
train = new_column(train)
train.head()

,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,...,employee1,employee2,ownerChange,revenue_change,16_n,17_n,OPM_change,16_liquid,17_liquid,liquidAsset_change
inst_id,,,,,,,,,,,,,,,,,,,,,
1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,1.000000e+00,3.961135e+09,2.033835e+09,...,63.0,64.0,same,-8.031824e+07,4.297848e+09,4.217530e+09,-8.031824e+07,0.245737,0.287104,0.041367
3,open,gyeongnam,32,19970401,410.0,general_hospital,4.170918e+10,6.750030e+09,3.398514e+10,1.911097e+10,...,802.0,813.0,same,3.172031e+09,3.197460e+10,3.495915e+10,2.984548e+09,0.236152,0.231855,-0.004297
4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,...,235.0,1.0,same,1.004522e+09,0.000000e+00,4.890383e+08,4.890383e+08,0.500000,0.693373,0.193373
7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,1.000000e+00,7.067740e+10,3.178605e+10,...,664.0,663.0,same,5.649006e+09,6.685834e+10,7.250734e+10,5.649006e+09,0.219402,0.231970,0.012568
9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,1.000000e+00,4.765605e+10,2.446078e+10,...,207.0,197.0,same,9.607370e+08,4.808280e+10,4.904354e+10,9.607370e+08,0.095392,0.126381,0.030989


In [78]:
print(train.isnull().sum())
#결측치 없어진거 확인

OC                    0
sido                  0
sgg                   0
openDate              0
bedCount              5
                     ..
17_n                  0
OPM_change            0
16_liquid             0
17_liquid             0
liquidAsset_change    0
Length: 64, dtype: int64
